# Currency risks for Swiss residents


## CMD: A Short Summary

If one wants to easily replicate the main findings of this paper, they might execute the command lines below (from the root folder). Later, we'll explore each part in further detail and explore the more nuanced analysis allowed by our implementations.

In [ ]:
:: Setup
setup.bat

:: Data retrieval and processing
python src/data/fred_data_retrieval.py
python src/data/oecd_data_retrieval.py
python src/data/fred_oecd_data_concatenation.py

:: Analysis, modeling and reporting
Rscript src/models/CODE_FAMA_CURRENCY.R
python src/analysis/performances.py

:: LaTeX files compiling

## Data retrieval

We use FRED to gather ForEx and VIX monthly data through FRED's API. We additionally gather German and Italian 10y-bond yields.

We obtain short-term interest rates through an html request to the OECD database.

CAUTION: OECD data for Japan (JPY) is available only since April 2002. Thus, it's not recommended to use a start date previous to this one.

### FRED

In [ ]:
import pandas as pd
import src.data.fred_data_retrieval as fred

# Define a time horizon
start_date = pd.Timestamp('2002-04-01')
end_date = pd.Timestamp('2023-10-31')

# Enter FRED API key
api_key = '41641234dac871704cd09b2b8dd163ac'

# Call to download function
fred.download(start_date=start_date,
              end_date=end_date,
              api_key=api_key)


### OECD

In [ ]:
import src.data.oecd_data_retrieval as oecd

oecd.download(start_date=start_date,
              end_date=end_date)

### Merge

In [ ]:
import src.data.fred_oecd_data_concatenation as merge
merge.merge()

### Load data

Now, we'll open the downloaded monthly data at a pandas Data Frame:

In [ ]:
df = pd.read_csv('data/processed/monthly_data.csv')

# Standardize data handling with pandas Timestamp
df['Date'] = df['Date'].map(lambda x: pd.Timestamp(x))
df.set_index('Date', inplace=True)

# Visualize the requested data
df.head(6)[['USD', 'GBP', 'EUR', 'VIX', 'USDi', 'GBPi', 'EURi', 'IT_minus_DE']]

## Analysis

### Regression results
We use R to compute the (log) forward rates and returns, and subsequently perform the linear regression in analysis.

The following command line reproduces the regression and publishes the results onto the LaTeX report, accessible at latex/paper.

### Installing IRkernel on the JupyterNotebook


First need to install and load the interactive kernel, in order to run code. In order to use the R code into the jupiterNotebook use the commande %%R.

In [ ]:
!pip install rpy2 -q


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
source("src/models/CODE_FAMA_CURRENCY.R")

In [ ]:
%%R
str(monthly_data)

With the str() command we can check all the variables, and source() allows use to make sure that we use the right folder. 

### Printing the regression results

The regression contains the coefficients for each parameters and the Durwin-Watson test. 


In [ ]:
%%R

print(regression_results)

The table provides an overview for the coefficient without the significance.

In [ ]:
%%R


print(dw_tests)


The Durbin-Watson test is great to check autocorrelation on the residuals of regressions. If the results are not significant, then there is no evidence for a "missing", non-explained autocorrelation in such data.

### Robustness test
Including the IT-DE spread into the regression:

First, let's plot such spread in the time frame in analysis:

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df['IT_minus_DE'].plot(xlabel='Time',
                       ylabel='pp',
                       title='IT-DE 10-year bond spread')
plt.savefig('latex/figures/it_de_spread.jpg')

In [ ]:
%%R
stargazer(models_list_Robust1, align = TRUE, type = "text", out = "latex/tables/regression_table_EUR_USD_JPY_R.tex")


In [ ]:
%%R
stargazer(models_list_Robust2, align = TRUE, type = "text", out = "latex/tables/regression_table_GBP_AUD_CAD_R.tex")


In [ ]:
%%R
stargazer(models_list_Robust3, align = TRUE, type = "text", out = "latex/tables/regression_table_NZD_SEK_NOK_R.tex")


TED spread robustness check

In [ ]:
%%R
stargazer(models_list_RobustTED1, align = TRUE, type = "text", out = "latex/tables/regression_table_EUR_USD_JPY_R.tex")


In [ ]:
%%R
stargazer(models_list_RobustTED2, align = TRUE, type = "text", out = "latex/tables/regression_table_GBP_AUD_CAD_RTED.tex")


In [ ]:
%%R
stargazer(models_list_RobustTED3, align = TRUE, type = "text", out = "latex/tables/regression_table_NZD_SEK_NOK_RTED.tex")


### FX rates in CHF

The following plot reproduces the FX rates for the specified countries list:

In [ ]:
import src.analysis.performances as perf

currencies_list = ['USD', 'CAD', 'GBP', 'EUR', 'AUD', 'NZD', 'JPY', 'NOK', 'SEK']
currencies_sublist = ['USD', 'GBP', 'JPY']
df2 = perf.build_df(currencies_list)
returns = perf.compute_returns(df2, currencies_list=currencies_list)

In [ ]:
perf.plot_performances(returns)

### Excess returns on Carry FX strategy

We also analyze the profitability and risks associated with investing in the short-term rates in each of the other G-10 currencies, from the standpoint of a Swiss investor.

In [ ]:
carry_fx_xs_returns = perf.compute_carry_fx_xs_returns(df2, currencies_list=currencies_list)
perf.plot_performances(carry_fx_xs_returns)

### Performance indicators

For each of the considered strategies, one might compute historical performance measures such as the Sharpe ratio, the average annualized return and the average and maximum drawdowns through the following code. It's also possible to make drawdown plots, histograms of returns and autocorrelation bar plots.

In [ ]:
perf.plot_return_histograms(returns, currencies_list=currencies_sublist)
perf.plot_return_histograms(carry_fx_xs_returns, currencies_list=currencies_sublist)

perf.plot_autocorrelations(carry_fx_xs_returns[currencies_sublist], currencies_list=currencies_sublist)

We've just verified that autocorrelations are negative for a 1-month horizon, for the three countries. Let's analyze this further.

In [ ]:
corr = perf.get_autocorrelations(excess_returns=carry_fx_xs_returns[:2018-12-31], currencies_list=currencies_list)
corr

In [ ]:
# Visualization of lag-1 autocorrelations
import matplotlib.pyplot as plt
plt.bar(corr.columns, corr.loc[1]); plt.title('1-month FX autocorrelations'); plt.show()

In [ ]:
# Visualization of mean-reversion persistence
corr.mean(1).cumsum().plot()
plt.title('Average cumsum of autocorrelations over G10 currencies')
plt.xlabel('Lag (months)')
plt.ylabel('corr')
plt.show()

The results above lead us to considering the evaluation of a mean-reversion FX strategy. We weight each currency proportionally to its autocorrelation.

In [ ]:
positions = ((1 - 2 * (carry_fx_xs_returns>0)).shift() * (-corr.loc[1]))
positions

In [ ]:
mean_reversion_returns = ((2* (carry_fx_xs_returns>0) - 1).shift() * corr.loc[1]) * carry_fx_xs_returns

In [ ]:
# Histogram of returns
mean_reversion_returns.sum(1).hist(bins=25)
plt.xlabel('return')
plt.ylabel('frequency')
plt.show()

In [ ]:
(mean_reversion_returns.sum(1)+1).cumprod().plot()
plt.title('Mean Reversion: Performance')
plt.show()

Computing the strategy's Sharpe ratio:

In [ ]:
print(f'Sharpe: {perf.sharpe(mean_reversion_returns.sum(1))}')

Additionally, let's compute the Sharpe Ratios on the Carry FX strategies over the G10 currencies:

In [ ]:
plt.bar(perf.sharpe(carry_fx_xs_returns).sort_values().index,
        perf.sharpe(carry_fx_xs_returns).sort_values())
plt.title('Sharpe ratio on Carry FX strategy')
plt.xlabel('Currency')
plt.ylabel('Sharpe')
plt.show()

### Covariance Matrix
We find out that the AFX values from the regression roughly match, in their order, the principal component of the empirical covariance matrix.
The code below computes and displays such component.

In [ ]:
# Eigenvalues
import numpy as np
np.linalg.eig(perf.get_correlation_matrix(carry_fx_xs_returns,
                      currencies_list=['USD', 'CAD', 'GBP', 'EUR', 'AUD', 'NZD', 'JPY', 'NOK', 'SEK']))[0]

In [ ]:
perf.plot_eigenvector_components(perf.get_correlation_matrix(carry_fx_xs_returns,
                                 currencies_list=currencies_list),
                                 currencies_list=currencies_list)
